In [1]:
import math
import pickle
import optuna

import numpy as np

from collections import defaultdict

#scikit-learn
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from optuna.samplers import RandomSampler


In [2]:
# import data
oer_desc_data = pickle.load(open("racs_and_desc/data_mc3_lc0.p", "rb"))

oer_desc_vals = []
racs = []
catalyst_names = []

for name in oer_desc_data:
    oer_desc_vals.append(oer_desc_data[name][0])
    racs.append(oer_desc_data[name][1])
    catalyst_names.append(name)
racs = np.asarray(racs)

In [3]:
from sklearn.model_selection import KFold
#Step 1. Define an objective function to be maximized.
def objective(trial):
    # Define set of models we will study.
    regressor_name = trial.suggest_categorical("regressor", ["SVR", "RandomForest", "KRR_RBF", "KRR_Linear"])
    
    # Step 2. Setup values for the hyperparameters:
    if regressor_name == "SVR":
        epsilon = trial.suggest_float("epsilon", 1e-3, 1, log=True)
        regularization = trial.suggest_float("reg_svr", 1e-3, 10, log=True)
        regressor_obj = SVR(epsilon=epsilon, C=regularization)
    elif regressor_name == "RandomForest":
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        regressor_obj = RandomForestRegressor(
            max_depth=rf_max_depth, n_estimators=rf_n_estimators, random_state=42
        )
    elif regressor_name == "KRR_RBF":
        krr_alpha = trial.suggest_float("krr_alpha", 0.01, 1000)
        regressor_obj = KernelRidge(kernel="rbf", alpha=krr_alpha)
        
    elif regressor_name == "KRR_Linear":
        krr_l_alpha = trial.suggest_float("krr_l_alpha", 0.01, 1000)
        regressor_obj = KernelRidge(kernel="linear", alpha=krr_l_alpha)

    # Step 3: Scoring method:
    preds = []
    test_vals = []
    errs = []
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(racs, oer_desc_vals):
        sc = StandardScaler()
        X_train, X_test = racs[train_index], racs[test_index]
        y_train, y_test = np.asarray(oer_desc_vals)[train_index], np.asarray(oer_desc_vals)[test_index]
        scaled_X_train = sc.fit_transform(X_train)
        scaled_X_test = sc.transform(X_test)

        model = regressor_obj

        model.fit(scaled_X_train, y_train)

        pred = model.predict(scaled_X_test,)
        preds.append(pred[0])
        test_vals.append(y_test[0])
    accuracy_rmse = math.sqrt(mean_squared_error(np.asarray(test_vals), np.asarray(preds)))
    return accuracy_rmse

# Step 4: Running it, we need a RandomSample with a seed so that the results are deterministic, and match
# the values provided in the communication, but we have found the same errors (although different hyperparams)
# using different seeds.
study = optuna.create_study(direction="minimize", sampler=RandomSampler(seed=42))
study.optimize(objective, n_trials=100)

[I 2021-09-02 19:25:16,222] A new study created in memory with name: no-name-9aabbf69-836e-466b-b05f-08bb8b9464d2
[I 2021-09-02 19:26:04,193] Trial 0 finished with value: 0.10061185345977103 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 164, 'rf_max_depth': 2}. Best is trial 0 with value: 0.10061185345977103.
[I 2021-09-02 19:26:22,471] Trial 1 finished with value: 0.08358021671002666 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 30, 'rf_max_depth': 30}. Best is trial 1 with value: 0.08358021671002666.
[I 2021-09-02 19:26:24,290] Trial 2 finished with value: 0.11333031998784011 and parameters: {'regressor': 'SVR', 'epsilon': 0.008179499475211672, 'reg_svr': 0.12561043700013558}. Best is trial 1 with value: 0.08358021671002666.
[I 2021-09-02 19:26:26,149] Trial 3 finished with value: 1.2463146376045744 and parameters: {'regressor': 'KRR_RBF', 'krr_alpha': 292.1517270887328}. Best is trial 1 with value: 0.08358021671002666.
[I 2021-09-02 19:26:27,704

[I 2021-09-02 19:44:04,258] Trial 36 finished with value: 0.2840237440472145 and parameters: {'regressor': 'SVR', 'epsilon': 0.49620663324183856, 'reg_svr': 0.2665116749055355}. Best is trial 9 with value: 0.07869925652682017.
[I 2021-09-02 19:44:05,213] Trial 37 finished with value: 1.0810984277654896 and parameters: {'regressor': 'KRR_RBF', 'krr_alpha': 160.81644333698446}. Best is trial 9 with value: 0.07869925652682017.
[I 2021-09-02 19:48:00,867] Trial 38 finished with value: 0.09150333298559964 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 715, 'rf_max_depth': 3}. Best is trial 9 with value: 0.07869925652682017.
[I 2021-09-02 19:48:01,504] Trial 39 finished with value: 1.532110590785897 and parameters: {'regressor': 'KRR_Linear', 'krr_l_alpha': 657.6163161714204}. Best is trial 9 with value: 0.07869925652682017.
[I 2021-09-02 19:48:04,296] Trial 40 finished with value: 0.08346599721717116 and parameters: {'regressor': 'SVR', 'epsilon': 0.005394720267647736, 're

[I 2021-09-02 20:19:13,777] Trial 73 finished with value: 1.531433328643805 and parameters: {'regressor': 'KRR_Linear', 'krr_l_alpha': 389.20778671737577}. Best is trial 9 with value: 0.07869925652682017.
[I 2021-09-02 20:24:44,405] Trial 74 finished with value: 0.08254744702842606 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 951, 'rf_max_depth': 28}. Best is trial 9 with value: 0.07869925652682017.
[I 2021-09-02 20:26:39,678] Trial 75 finished with value: 0.08248839857690024 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 335, 'rf_max_depth': 12}. Best is trial 9 with value: 0.07869925652682017.
[I 2021-09-02 20:28:06,240] Trial 76 finished with value: 0.1006654132817919 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 499, 'rf_max_depth': 2}. Best is trial 9 with value: 0.07869925652682017.
[I 2021-09-02 20:28:06,631] Trial 77 finished with value: 0.9784798927231352 and parameters: {'regressor': 'KRR_RBF', 'krr_alpha': 117.07584575